In [1]:
device = 'cuda:0'
mode = 'simba_electronic'
# mode = 'incontext_simba'
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [7]:
import numpy as np
a = np.load('/mnt/gestalt/home/lonian/datasets/MusicCaps/dac_token/_-kssA-FOzU.npy')

In [8]:
a.shape

(9, 862)

In [2]:
import datasets
datasets_200 = datasets.load_from_disk('/mnt/gestalt/home/lonian/datasets/MusicCaps/eval_sub_200')

/home/lonian/anaconda3/envs/mamba_hf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


13

In [25]:
datasets_200[datasets_200['ytid'].index('MY0PsDE3xHs')]

{'ytid': 'MY0PsDE3xHs',
 'start_s': 30,
 'end_s': 40,
 'audioset_positive_labels': '/m/015lz1,/m/04rlf,/m/05zppz,/m/064t9,/t/dd00003',
 'aspect_list': "['female singer', 'medium fast tempo', 'emotional', 'contemporary pop', 'upbeat', 'groovy rhythm', 'dance music', 'dance pop', 'club music', 'electro pop', 'synth pop', 'ambient', 'electronic music', 'synthesiser articulation', 'digital drumming', 'keyboard arrangement', 'electric guitar riff', 'vocal backup']",
 'caption': 'A female singer sings soft pop songs. The tempo is medium fast with electronically arranged music, electric guitar harmony, digital drum beats, and melodious vocal backup. The song is simple, soft, emotional, upbeat, catchy, with a dance groove. This song is an Electro Pop song.',
 'author_id': 7,
 'is_balanced_subset': True,
 'is_audioset_eval': True}

In [6]:
for i in loader:
    # if os.path.isfile(os.path.join(save_path, '{}.npy'.format(i['ytid'][0]))):
    #     continue
    text_prompts = i['caption']
    print(i)
    break

{'ytid': ['-bgHkxwoliw', '-kpR93atgd8', '-wymN80CiYU', '07xGXxIHOL4', '0PMFAO4TIU4', '0TV9zvfwFhs', '0fiOM---7QI', '0i8VM_EooCs', '0jFQ21A6GRA', '1ACn3u5UnBw'], 'start_s': tensor([ 30,  30, 130, 110,  30,  30, 140,  30,  30,  30]), 'end_s': tensor([ 40,  40, 140, 120,  40,  40, 150,  40,  40,  40]), 'audioset_positive_labels': ['/m/04rlf,/m/04wptg', '/m/015y_n,/m/0164x2,/m/04rlf,/m/06cqb', '/m/04rlf,/t/dd00031', '/m/04rlf,/m/04wptg', '/m/03_d0,/m/04rlf', '/m/04rlf,/t/dd00003,/t/dd00006', '/m/04rlf,/m/06j64v', '/m/02lkt,/m/04rlf,/m/07gxw', '/m/04rlf,/m/06j64v', '/m/04rlf,/t/dd00003'], 'aspect_list': ["['pop', 'digital drums', 'e-guitar', 'keyboard melodies', 'male voice singing', 'higher register', 'bass', 'uptempo', 'positive atmosphere']", "['ska', 'saxophone', 'e-guitar', 'acoustic drums', 'percussion', 'keys', 'e-bass', 'fast tempo', 'uplifting energy']", "['male singer', 'backup singers', 'guitar lead', 'medium tempo', 'steady drumming rhythm', 'string section playing rhythmically'

In [6]:
print(coarse_token.shape, fine_noise.shape)
d = torch.cat( (coarse_token, fine_noise) , 1)
print(d.shape)
print(d[:, :8, :]==coarse_token)

torch.Size([1, 8, 2588]) torch.Size([1, 64, 2588])
torch.Size([1, 72, 2588])
tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]]])


In [2]:
import sys
# pytorch
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.nn.functional as F
from pl_model import Text_Mmamba_pl
# import lightning as L
# from lightning.pytorch.callbacks.early_stopping import EarlyStopping
# others
from glob import glob
import numpy as np
# import os
import json
from tqdm import tqdm
import math
# import argparse
from transformers import T5EncoderModel, T5Tokenizer
# from text_simba import MB_Dataset
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# torch.multiprocessing.set_start_method('spawn')
from utils import *

# mode = 'incontext_simba'
# print(mode)

def create_logger(logger_file_path, name=None):
    import time
    import logging
    
    if not os.path.exists(logger_file_path):
        os.makedirs(logger_file_path)
    if name is not None:
        log_name = '{}.log'.format(time.strftime('%Y-%m-%d-%H-%M'))
    else:
        log_name = '{}.log'.format(name)
    final_log_file = os.path.join(logger_file_path, log_name)

    logger = logging.getLogger()  # 设定日志对象
    logger.setLevel(logging.INFO)  # 设定日志等级

    file_handler = logging.FileHandler(final_log_file)  # 文件输出
    console_handler = logging.StreamHandler()  # 控制台输出

    # 输出格式
    formatter = logging.Formatter(
        "%(asctime)s %(levelname)s: %(message)s "
    )

    file_handler.setFormatter(formatter)  # 设置文件输出格式
    console_handler.setFormatter(formatter)  # 设施控制台输出格式
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    return logger


model_path='/mnt/gestalt/home/lonian/mamba/model/ckpts/simba_9_incontext/lightning_logs/version_4/checkpoints/epoch=150-step=64500.ckpt'
folder_name='description'
subfolder_name='prefix_simba_9'

config_path = model_path[::-1].split('/', 4)[-1][::-1]+'/config.json'
with open(config_path) as f:
    config = json.load(f)
model = Text_Mmamba_pl.load_from_checkpoint(model_path, config)
model.eval()
model.freeze()
# folder_name = 'musicgen_baseline'
save_path = f'/mnt/gestalt/home/lonian/mamba/exp_results/{folder_name}/{subfolder_name}/dac_token'
os.makedirs (save_path, exist_ok=True)

logger = create_logger(f'/mnt/gestalt/home/lonian/mamba/exp_results/{folder_name}/{subfolder_name}')

logger.info(f'Is incontext: {config['model']['is_incontext']}')
logger.info(f'Attention layers: {config['model']['self_atten_layers']}')
logger.info(f'Is pure mamba: {config['model']['is_pure_mamba']}')
logger.info(model_path)

# model = Text_Mmamba_pl.load_from_checkpoint("/mnt/gestalt/home/lonian/mamba/model/ckpts/simba_9_incontext/lightning_logs/version_4/checkpoints/epoch=150-step=64500.ckpt", config)

# model.eval()
# model.freeze()
# folder_name = 'description'
# save_path = f'/mnt/gestalt/home/lonian/mamba/exp_results/{folder_name}/{mode}/dac_token'
# os.makedirs (save_path, exist_ok=True)

# logger = create_logger(f'/mnt/gestalt/home/lonian/mamba/exp_results/{folder_name}/{mode}', name=mode)

# logger.info(f'Is incontext: {config['model']['is_incontext']}')
# logger.info(f'Attention layers: {config['model']['self_atten_layers']}')
# logger.info(f'Is pure mamba: {config['model']['is_pure_mamba']}')
# # "self_atten_layers": [],
# #         "is_incontext": true,
# #         "is_pure_mamba": false

/home/lonian/anaconda3/envs/mamba_hf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2025-02-13 12:22:50,299 INFO: Is incontext: True 
2025-02-13 12:22:50,301 INFO: Attention layers: [] 
2025-02-13 12:22:50,302 INFO: Is pure mamba: Fa

In [3]:
L = 2588//3
with torch.autocast(device_type="cuda", dtype=torch.float32):
    with torch.no_grad():
        device = 'cuda'
        description = ['A vibrant and catchy melody drives this upbeat pop track, featuring smooth vocals layered over a backdrop of rhythmic claps, bright synths, and a steady drumbeat. The song exudes positivity and is perfect for a sunny day or a lively party.', 
                    'This mellow jazz piece blends a walking bassline with intricate piano improvisations and the soft hum of a saxophone solo. Its laid-back tempo and smooth swing rhythm create a cozy, sophisticated atmosphere ideal for a relaxed evening.', 
                    'A high-energy EDM track with thumping bass drops, euphoric synth leads, and an infectious rhythm. The buildup and release of tension keep the crowd on their feet, making it the ultimate choice for a late-night dance party.', 
                    'An explosive rock anthem powered by crunchy electric guitar riffs, a pulsating bassline, and dynamic drum fills. The raw, emotive vocals bring energy and intensity, making it an ideal soundtrack for a road trip or an adrenaline-pumping workout.', 
                    'A delicate and expressive piano solo, weaving a gentle melody with flowing arpeggios. The piece captures a contemplative mood, evoking imagery of a serene landscape at sunrise, perfect for moments of quiet reflection.',
                    'An experimental electronic piece blending glitchy beats, atmospheric textures, and robotic vocal effects. The track evolves through intricate soundscapes and pulsating basslines, creating a futuristic and immersive sound journey.']
        
        # description = ['A rhythmic electronic track with steady beats, shimmering synths, and a deep, pulsing bassline, creating an immersive, futuristic vibe.',
        #                'A dreamy electronic composition filled with soft pads, airy melodies, and a gentle, flowing rhythm that feels ethereal and calming.',
        #                'A high-energy electronic song featuring fast beats, glitchy effects, and bright synth leads that build an exciting, dynamic mood.',
        #                'A minimalist electronic piece with repetitive beats, subtle textures, and a hypnotic, looping melody that feels introspective and atmospheric.',
        #                'A vibrant electronic tune with layered arpeggios, sparkling synths, and a bouncy bass that creates an uplifting and playful sound.']
        
        # print(len(i['ytid']))
        prompt_seq = model(description=description, length=L, g_scale=3)
        print(prompt_seq.shape)

        for b in range(len(description)//2):
            gen_id = len(glob(f'{save_path}/*.npy')) + 1
            np.save(os.path.join(save_path, '{}.npy'.format(f'{gen_id}_{description[b].replace(' ', '_')}')), prompt_seq[b, :, :L])

(6, 9, 862)


In [5]:
model.config['model']['codec_layer']

1

In [4]:
len(description)

8

In [5]:
description

['Pop music',
 'Jazz',
 'Electronic Dance Music',
 'Rock music with heavy metal style',
 '',
 '',
 '',
 '']